In [1]:
import cv2
from PIL import Image
import numpy as np
import os
import random

global id
id = 0

# Data collector
def generate_dataset(id):
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            cropped_face = img[y:y+h, x:x+w]
            return cropped_face

    cap = cv2.VideoCapture(0)
    img_id = 0

    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id += 1
            face = cv2.resize(face_cropped(frame), (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            # Data augmentation: random flips
            if random.random() < 0.5:
                face = cv2.flip(face, 1)

            # Histogram equalization
            equalized_face = cv2.equalizeHist(face)
            cv2.imwrite(f"data/user.{id}.{img_id}.jpg", equalized_face)

            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Cropped face", face)

        if cv2.waitKey(1) == 13 or img_id == 200:
            break

    cap.release()
    cv2.destroyAllWindows()

    print("Collecting samples is completed....")

# Training classifier
def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id_c = int(os.path.split(image)[1].split(".")[1])
        faces.append(imageNp)
        ids.append(id_c)

    ids = np.array(ids)

    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier1.xml")

# Evaluate accuracy
def evaluate_accuracy(data_dir, clf):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    actual_ids = []
    predicted_ids = []
    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    for image in path:
        img = Image.open(image).convert('L')
        image_np = np.array(img, 'uint8')
        id_c = int(os.path.split(image)[1].split(".")[1])

        faces.append(image_np)
        actual_ids.append(id_c)

        gray = cv2.cvtColor(image_np, cv2.COLOR_GRAY2BGR)
        features = faceCascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in features:
            gray_roi = gray[y:y + h, x:x + w]
            gray_roi = cv2.cvtColor(gray_roi, cv2.COLOR_BGR2GRAY)
            id_p, confidence = clf.predict(gray_roi)
            predicted_ids.append(id_p)

    correct_predictions = sum(1 for actual, predicted in zip(actual_ids, predicted_ids) if actual == predicted)
    print(correct_predictions)
    accuracy = correct_predictions / len(actual_ids) * 100

    return accuracy

# Caller for accuracy function
def accuracy_caller():
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier1.xml")
    accuracy = evaluate_accuracy("data", clf)
    print(f"Accuracy: {accuracy:.2f}%")

# Detecting the faces
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    id_p = -1

    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
        id_p, pred = clf.predict(gray_img[y:y+h, x:x+w])
        confidence = int(100 * (1 - pred / 300))

        if confidence > 75:
            cv2.putText(img, str(id_p), (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            cv2.putText(img, "UNKNOWN", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)

    return [img, id_p]

# Caller for face detection
def detect():
    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier1.xml")
    video_capture = cv2.VideoCapture(0)

    while True:
        ret, img = video_capture.read()
        l1 = draw_boundary(img, faceCascade, 1.3, 6, (255, 255, 255), "Face", clf)
        cv2.imshow("face Detection", l1[0])

        if cv2.waitKey(1) == 13:
            break

    video_capture.release()
    cv2.destroyAllWindows()
    return l1[1]

# Normal functionality
while True:
    print("1. Add face\n2. Train\n3. Detect face\n4. Exit")
    x1 = int(input("Enter your choice: "))

    if x1 == 1:
        id += 1
        generate_dataset(id)
    elif x1 == 2:
        train_classifier("data")
        print("Training done....")
        accuracy_caller()
    elif x1 == 3:
        id_out = detect()
        print(id_out)
    else:
        break

1. Add face
2. Train
3. Detect face
4. Exit
1. Add face
2. Train
3. Detect face
4. Exit
Training done....
93
Accuracy: 23.25%
1. Add face
2. Train
3. Detect face
4. Exit
1
1. Add face
2. Train
3. Detect face
4. Exit
